In [1]:
#抓取新聞連結
def getlink(script):
    arry = [] 
    global count
    global pkcount 
    apl = 'apl'
    soup = BeautifulSoup(res.text)
    for a_tag in soup.select('#result h2 a'):
        alink = a_tag.get('href').strip()
        arry.append(alink)
        for elelink in arry:
            num = str(pkcount)
            if pkcount < 10:
                num = str(0) * 4 + num
            elif pkcount < 100:     
                num = str(0) * 3 + num
            elif pkcount < 1000:     
                num = str(0) * 2 + num
            elif pkcount < 10000:     
                num = str(0) * 1 + num
            pk = apl + num
        links = {'link':alink,'area':area,'page':page,'count':count,'pk':pk}
        linkary.append(links)
        count += 1
        pkcount += 1

In [ ]:
import requests
from datetime import *
from bs4 import BeautifulSoup
import time

#12個行政區['中正區','大同區','中山區','松山區','大安區','萬華區','信義區','士林區','北投區','內湖區','南港區','文山區']
ary = ['中正區','大同區','中山區','松山區','大安區','萬華區','信義區','士林區','北投區','內湖區','南港區','文山區']
links = {'link':'','area':'','page':'','count':'','pk':''}
linkary =[]
    
#系統時間------------------------------
today = date.today()
year = today.year
month = today.month
day = today.day
if month < 10:
    month = "0" + str(month)
if day < 10:
    day = "0" + str(day)
year = str(year)
month = str(month)
day = str(day)
#--------------------------------------

headers={
'Origin':'http://search.appledaily.com.tw',
'Referer':'http://search.appledaily.com.tw/appledaily/search',
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36' 
}
pkcount = 1    # pkcount = 1
for area in ary:
    count = 1
    for page in range(1,12):
        payload = {
        'searchMode':'',
        'searchType':'text',
        'ExtFilter':'',
        'sorttype':'1',
        'keyword':area,
        'rangedate':'[20030502 TO'+year+month+day+'999:99'']',
        'totalpage':'',
        'page':page
        }

        res = requests.post('http://search.appledaily.com.tw/appledaily/search',headers=headers,data=payload)
        script = res.text
        getlink(script)

In [ ]:
import pandas
data1 = pandas.DataFrame(linkary)
data1

In [48]:
#輸出成CSV
data1.to_csv('apl_link.csv',encoding='utf-8')

NameError: name 'data1' is not defined

In [65]:
#讀取CSV檔
import csv
linkary = []
with open('apl_link.csv',) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        linkary.append(row)

In [66]:
#取得人氣
def getlike(soup): 
    if len(soup.select('.urcc a')) > 0:
        reele = soup.select('.urcc a')[0].text
        print soup.select('.urcc a')
        rdrx = re.search('(\d+)',reele)
        likes = rdrx.group(1)
    else:
        likes = "0"
    return likes 

In [67]:
#抓取新聞內文
import requests
import time
import re  # --
from bs4 import BeautifulSoup

headers = { 
'Origin':'http://search.appledaily.com.tw',
'Referer':'http://search.appledaily.com.tw/appledaily/search',
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36'
}

for ele in linkary:
    url = ele['link']
    text = ''
    while True:
        try:
            res = requests.get(url,headers=headers)
            res.text
            soup = BeautifulSoup(res.text)
            break
        except:
            time.sleep(5)
            
    try:
        ele['title'] = soup.select('hgroup #h1')[0].text.strip()
        ele['date'] = soup.select('.gggs time')[0]['datetime']
        for p_tag in soup.select('#summary'):
            text += p_tag.text.strip()
        ele['content'] = text
        ele['likes'] = getlike(soup)
        
    except:
        print "This link has some problems!"
        print ele['link'], ele['area'], ele['count']

This link has some problems!
http://www.appledaily.com.tw/appledaily/article/property/20151011/36830793/applesearch/名人投資GiGi林如琦房產與副業多元投資 中山區 42
This link has some problems!
http://www.appledaily.com.tw/appledaily/article/property/20151011/36830793/applesearch/名人投資GiGi林如琦房產與副業多元投資 大安區 56


In [68]:
#新聞內文進入DataFrame
import pandas
data2 = pandas.DataFrame(linkary)

# 移除多餘的 column
# del data2['column name']
apl_ok = data2.drop(data2.columns[[0]], axis=1)
apl_ok

area                                            content count  \
0     中正區  第13屆台灣同志大遊行今天登場，力挺婚姻平權的第三勢力也紛紛支持，時代力量、綠社盟都分別帶隊...     1   
1     中正區  雙層店面好運用，價格水漲船高，房仲統計近1年雙北店面交易實價，台北市1樓店面年減0.6%、新...     2   
2     中正區  台北市衛生局今公布新增1例本土登革熱個案。新增本土個案一名為中正區26歲男子，本月16日至1...     3   
3     中正區                                                        4   
4     中正區                                                        5   
5     中正區  （更新：新增嫌搶匪所乘計程車、逃逸路線）博愛特區驚傳銀行搶案！今天下午2時許，一名男子跑到台...     6   
6     中正區  國產署今北中南同步進行地上權土地招標，13筆僅標出2筆。 這次招標面積共38162.72平方...     7   
7     中正區  亮槍恐嚇被判刑2月！基隆市33歲陳姓男子，開車行經中正區新豐街，因不滿後方車輛跟車太近，似乎...     8   
8     中正區  本內容由公庫提供文／法律扶助基金會時間：105/10/23（五）早上09：00地點：台北地方...     9   
9     中正區  本內容由公民監督國會聯盟提供第九屆立法委員選舉，除了傳統國、民兩大黨，更有許多小黨紛紛投入參...    10   
10    中正區  台北市受打房衝擊，蛋黃真的破了！信義房屋統計內部中古屋成交資訊，將今年第3季和去年房價高峰的...    11   
11    中正區  近來許多香港人移居台灣，這股置產風也吹到地上權產品，位台北市中正區的地上權案「將捷SOLO」...    12   
12    中正區  南機場公辦都更案今年7月再度重啟，點燃北市都更話題。近日更因台北市長柯文哲能否於任內完成南機...    13   
13    中正區  滙豐(台灣)商業銀「東門分行」全新開幕，營業時間延長至下午5 點，滿足客戶財富管理諮詢需求。...    14   
14    中正區  繼公布淹水、住宅竊盜熱點後，台北市長柯文哲昨宣布，將再公布性侵熱點。北市警察局昨說，將改良現...    15   
15    中正區                                                       16   
16    中正區                                                       17   
17    中正區  都更案的整合開發曠日廢時，繼寶舖建設、勤耕建設曾在接待中心或待都更的舊屋打造咖啡館，近期名冠...    18   
18    中正區  北市衛生局今公布新增3例本土登革熱確診個案，新增3例個案分別居住於信義區、南港區及中正區，3...    19   
19    中正區  台北市雖為首善之都，但房價也高到嚇人，據今年1~8月實價登錄資料，台北市住宅每坪單價63.3...    20   
20    中正區                                                       21   
21    中正區  本內容由公庫提供圖文／好勁稻工作室臺北市議員郭昭巖繼於本週一召開「嘉禾新村遲未拆除」會勘後，...    22   
22    中正區                                                       23   
23    中正區  【特別企劃】重劃區向來是房市寵兒，因為具備完善規劃、品質佳的生活及未來發展潛力高等要素，而受...    24   
24    中正區                                                       25   
25    中正區  10/24、10/31僅限兩場！全台遍地開花【行銷專題企劃】蘋果日報行銷業務部自去年舉辦「蘋...    26   
26    中正區                                                       27   
27    中正區                                                       28   
28    中正區  （更新：新增影片）太恐怖了！強颱杜鵑橫掃北台灣，許多建築物外牆的招牌不堪強風吹拂搖搖欲墜，北...    29   
29    中正區  落跑名媛吳麗玲的「元大一品苑」金拍第二拍標脫，拍賣底價2億1016萬元，總面積178.98坪...    30   
...   ...                                                ...   ...   
1290  文山區  北市受打房影響，房價呈下修，但店面因有觀光商機加持，不跌反漲，據全國不動產統計，今年共9行政...    81   
1291  文山區  有媒體報導，台北市立動物園園長金仕謙遭園內員工指控，將2個月前送至園內的穿山甲私下拿來做實驗...    82   
1292  文山區  作者：孫榮富（國會辦公室研究員）到了上海雙城論壇的柯文哲幽了自己一默說：一年前又有誰會想到我...    83   
1293  文山區  （新增：水利署最新淹水警戒）水利署剛發布淹水警戒，其中，台北市文山區為1級淹水警戒，另外台北...    84   
1294  文山區                                                       85   
1295  文山區                                                       86   
1296  文山區  1名罹患輕度智能障礙的吳姓男子，今年5月到北市文山區1戶民宅偷走1雙女用NIKE球鞋，被逮後...    87   
1297  文山區  台北市文山區1名老翁指責女鄰居半夜騎機車太吵，女鄰居的陳姓丈夫竟氣沖沖到老翁家理論，見老翁開...    88   
1298  文山區                                                       89   
1299  文山區                                                       90   
1300  文山區  即將農曆7月鬼門開，好房網統計近2年台北市墓地周邊住宅成交均價，發現大安區臥龍街今年單坪64...    91   
1301  文山區  （更新：淹水警戒範圍）受到西南風影響，今天午後有大雷雨，中央氣象局已發布大雨特報，水利署剛也...    92   
1302  文山區                                                       93   
1303  文山區                                                       94   
1304  文山區  自住客想晉升有殼族，該從哪裡著手？太平洋房屋統計實價登錄雙北市住宅今年以來的平均成交總價，台...    95   
1305  文山區  民眾李先生投訴指，昨天和今天到北市文山區萬隆一帶的堤外自行車道跑步，看到風災後被沖上岸的死魚...    96   
1306  文山區  今天凌晨1時許，1輛公車因路燈停電視線不佳，不慎撞上台北市文山區興隆路2段中央分隔島上尚未清...    97   
1307  文山區                                                       98   
1308  文山區  去年3月台北市某酒店女子在酒店內跟男客玩遊戲卻慘敗，喝下近3瓶威士忌，下班時爛醉搭莊男（47...    99   
1309  文山區  （更新：新增現場照片、短片）台北市文山區通往新店的一壽橋，今天早上６時許，有民眾運動時發現河...   100   
1310  文山區  （更新：北市府統計內容）台北市政府統計，蘇迪勒颱風造成北市主要災損至今下午1時止，總共案件數...   101   
1311  文山區  （更新：新增影片）蘇迪勒颱風肆虐北台灣過後，臉友Hui Hui Lin在臉書PO出一段影片「...   102   
1312  文山區  (更新：新增最新淹水即時)蘇迪勒颱風挾帶豪大雨，水利署發布台北市、新北市、基隆市、宜蘭縣、桃...   103   
1313  文山區  蘇迪勒颱風除了風強，降雨強度也大，造成全台多處河川水位暴漲，今天上午也傳出新北市烏來有1名男...   104   
1314  文山區  蘇迪勒颱風襲災情頻傳，中央氣象局發布海陸颱風警報，台北市12行政區自入夜後災情頻傳，路樹倒塌...   105   
1315  文山區  （更新：

In [69]:
#輸出完成品
apl_ok.to_csv('apl.csv',encoding='utf-8')

In [56]:
import requests
headers={
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36'
}
res = requests.get('http://www.appledaily.com.tw/appledaily/article/headline/20151104/36879965/applesearch/%E5%AA%9A%E7%99%BB%E5%B3%B0%E8%8E%8A%E9%9B%85%E6%B8%85%E5%A7%AA%E5%85%92%E8%BF%BD%E5%85%AC%E8%BB%8A%E7%AA%81%E6%98%8F%E5%80%92',headers=headers)
res.text

u'<!DOCTYPE html>\n<!--[if lt IE 7 ]> <html lang="zh-TW" class="ie6 ielt8"> <![endif]-->\n<!--[if IE 7 ]>    <html lang="zh-TW" class="ie7 ielt8"> <![endif]-->\n<!--[if IE 8 ]>    <html lang="zh-TW" class="ie8"> <![endif]-->\n<!--[if (gte IE 9)|!(IE)]><!--> <html lang="zh-TW"> <!--<![endif]-->\n<head>\n    <meta charset="utf-8" />\n    <title>\u5a9a\u767b\u5cf0 \u838a\u96c5\u6e05\u59ea\u5152 \u8ffd\u516c\u8eca\u7a81\u660f\u5012 | \u860b\u679c\u65e5\u5831</title>\n    <meta name="Title" content="\u5a9a\u767b\u5cf0 \u838a\u96c5\u6e05\u59ea\u5152 \u8ffd\u516c\u8eca\u7a81\u660f\u5012 | \u860b\u679c\u65e5\u5831"/>\n    <meta name="description" content="\u3010\u9673\u5049\u5468\u3001\u6234\u4e4b\u8056\u2571\u57fa\u9686\u5831\u5c0e\u3011\u5a9a\u767b\u5cf0\u96c6\u5718\u7e3d\u88c1\u838a\u96c5\u6e05\uff0c\u5176\u5c31\u8b80\u57fa\u9686\u7d93\u570b\u7ba1\u7406\u66a8\u5065\u5eb7\u5b78\u9662\u4e8c\u5e74\u7d1a\u7684\u59ea\u5152\uff0c\u6628\u4e0b\u8ab2\u5e36\u8457\u5973\u53cb\u8dd1\u8457\u8ffd\u516c\u

In [61]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text)

In [64]:

print soup.select('#toolbar a')

[<a class="smaller overcooked" href="javascript:doZoom(16)" id="smaller" mce_href="javascript:doZoom(16)" title="\u6700\u5c0f\u5b57\u578b">\u6700\u5c0f\u5b57\u578b</a>, <a class="middle overcooked" href="javascript:doZoom(18)" id="middle" mce_href="javascript:doZoom(18)" title="\u9810\u8a2d">\u9810\u8a2d</a>, <a class="bigger overcooked" href="javascript:doZoom(20)" id="bigger" mce_href="javascript:doZoom(20)" title="\u6700\u5927\u5b57\u578b">\u6700\u5927\u5b57\u578b</a>, <a class="function_icon clicked" target="_self" title="\u9ede\u95b1">\u4eba\u6c23(15132)</a>, <a class="function_icon forward" href="javascript:void(window.open('/appledaily/forward/sectionKey/headline/art_id/36879965/IssueID/20151104/Title/\u5a9a\u767b\u5cf0\u838a\u96c5\u6e05\u59ea\u5152\u8ffd\u516c\u8eca\u7a81\u660f\u5012','Forward','toolbar=no,width=470,height=320,directories=no,status=no,scrollbars=no,resize=no,menubar=no'));" target="_self" title="\u8f49\u5bc4">\u8f49\u5bc4(0)</a>, <a class="function_icon quote" 

In [28]:
if len(soup.select('.urcc a')) > 0:
    reele = soup.select('.urcc a')[0].text
    print "1"
    rdrx = re.search('(\d+)',reele)
    likes = rdrx.group(1)
elif len(soup.select('.fntss a')) > 0:
    print "2"
    reele = soup.select('.fntss a')[0].text
    rdrx = re.search('(\d+)',reele)
    likes = rdrx.group(0)
else:
    likes = "0"
    print likes

2


AttributeError: 'NoneType' object has no attribute 'group'